In [0]:
!pip install xgboost
!pip install numpy Cython
!pip install pymrmr

import io
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.pipeline import make_pipeline
import statistics
from sklearn.metrics import matthews_corrcoef
import pymrmr


hgg1=pd.read_csv("")#Path to csv
lgg1=pd.read_csv("")

lgg1["GRADE"]=0
hgg1["GRADE"]=1
hgg= hgg1.sample(frac=1, random_state=42).reset_index(drop=True)
lgg= lgg1.sample(frac=1, random_state=42).reset_index(drop=True)



auc_lr=[]
spe_lr=[]
sen_lr=[]
mcc_lr=[]
acc_lr=[]
f1_lr=[]
pre_lr=[]
rec_lr=[]
pre_lrt=[]
rec_lrt=[]
auc_lrt=[]
spe_lrt=[]
mcc_lrt=[]
sen_lrt=[]
acc_lrt=[]
f1_lrt=[]

master=pd.concat([lgg, hgg])
master= master.sample(frac=1, random_state=42).reset_index(drop=True)
master= pd.concat([master["GRADE"], master.drop("GRADE", axis=1)], axis=1)
Y=master["GRADE"]
master = (master - master.min()) / (master.max() - master.min())

cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=False)
for train_index, test_index in cv.split(master, Y):
    #print("Train Index: ", train_index, "\n")
    #print("Test Index: ", test_index)

    train, test = master.iloc[train_index], master.iloc[test_index]
    Y_train, Y_test= Y.iloc[train_index], Y.iloc[test_index]
    features=pymrmr.mRMR(train, 'MIQ', 43)
    print("Best Features: ", features)

    dataset=master[features]

    X_train=dataset.iloc[train_index]
    X_test=dataset.iloc[test_index]
    
    clf = XGBClassifier(n_estimators=450, max_depth=2, learning_rate=0.1)
    clf.fit(X_train, Y_train)

    pred = clf.predict(X_test)
    cm = metrics.confusion_matrix(Y_test, pred)
    TP = cm[1][1]
    TN = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]
    sen_l = (TP / float(TP + FN))
    spe_l = (TN / float(TN + FP))
    pre_l = precision_score(Y_test, pred)
    rec_l = recall_score(Y_test, pred)
    mcc_l = matthews_corrcoef(Y_test, pred)
    acc_l = accuracy_score(Y_test, pred)
    probs = clf.predict_proba(X_test)
    probs = probs[:, 1]
    auc_l = roc_auc_score(Y_test, probs)
    f1_l= f1_score(Y_test, pred)

    
    pred = clf.predict(X_train)
    
    cm = metrics.confusion_matrix(Y_train, pred)
    TP = cm[1][1]
    TN = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]
    sen_lt = (TP / float(TP + FN))
    spe_lt = (TN / float(TN + FP))
    pre_lt = precision_score(Y_train, pred)
    rec_lt = recall_score(Y_train, pred)
    mcc_lt = matthews_corrcoef(Y_train, pred)
    acc_lt = accuracy_score(Y_train, pred)
    probs = clf.predict_proba(X_train)
    probs = probs[:, 1]
    auc_lt = roc_auc_score(Y_train, probs)
    f1_lt = f1_score(Y_train, pred)


    auc_lr.append(auc_l)
    spe_lr.append(spe_l)
    sen_lr.append(sen_l)
    mcc_lr.append(mcc_l)
    acc_lr.append(acc_l)
    f1_lr.append(f1_l)
    rec_lr.append(rec_l)
    pre_lr.append(pre_l)

    pre_lrt.append(pre_lt)
    rec_lrt.append(rec_lt)
    auc_lrt.append(auc_lt)
    spe_lrt.append(spe_lt)
    sen_lrt.append(sen_lt)
    mcc_lrt.append(mcc_lt)
    acc_lrt.append(acc_lt)
    f1_lrt.append(f1_lt)


print("TRAIN:")
print("PRE:", statistics.mean(pre_lrt))
print("REC:", statistics.mean(rec_lrt))
print("SPE:", statistics.mean(spe_lrt))
print("MCC:", statistics.mean(mcc_lrt))
print("SEN:", statistics.mean(sen_lrt))
print("ACC:", statistics.mean(acc_lrt))
print("AUC:", statistics.mean(auc_lrt))
print("F1:", statistics.mean(f1_lrt))

print("TEST:")
print("PRE:", statistics.mean(pre_lr))
print("REC:", statistics.mean(rec_lr))
print("SPE:", statistics.mean(spe_lr))
print("MCC:", statistics.mean(mcc_lr))
print("SEN:", statistics.mean(sen_lr))
print("ACC:", statistics.mean(acc_lr))
print("AUC:", statistics.mean(auc_lr))
print("F1:", statistics.mean(f1_lr))